In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "jarnorantaharju"
os.environ['KAGGLE_KEY'] = "1c36d632e191d13cdda5450b1396f020"

In [20]:
import pandas as pd
import niimpy
from niimpy.exploration.eda import countplot
from niimpy.preprocessing import survey
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
api = KaggleApi()
api.authenticate()

In [4]:
api.dataset_download_files('dartweichen/student-life', path=".")
archive = zipfile.ZipFile('student-life.zip', 'r')
activity_data = []
for user_number in range(60):
    user = f"u{user_number:02}"
    try:
        csvfile = archive.open(f"dataset/sensing/activity/activity_{user}.csv")
        user_activity = pd.read_csv(csvfile)
        user_activity["user"] = user
        activity_data.append(user_activity)
    except:
        pass
activity_data = pd.concat(activity_data)

activity_data["activity"] = activity_data[" activity inference"].astype(object)
activity_data.set_index('timestamp',inplace=True)
activity_data.index = pd.to_datetime(activity_data.index, unit='s')

In [51]:
csvfile = archive.open(f"dataset/survey/PHQ-9.csv")
survey_data = pd.read_csv(csvfile)
survey_data = survey_data.rename(columns={'uid': 'user'})
survey_data = survey_data[survey_data["type"] == "pre"]
# survey_data = survey_data.set_index("user")

In [56]:
PHQ9_MAP = {
    'Little interest or pleasure in doing things': "PHQ9_1",
    'Feeling down, depressed, hopeless.': "PHQ9_2",
    'Trouble falling or staying asleep, or sleeping too much.': "PHQ9_3",
    'Feeling tired or having little energy': "PHQ9_4",
    'Poor appetite or overeating': "PHQ9_5",
    'Feeling bad about yourself or that you are a failure or have let yourself or your family down': "PHQ9_6",
    'Trouble concentrating on things, such as reading the newspaper or watching television': "PHQ9_7",
    'Moving or speaking so slowly that other people could have noticed. Or the opposite being so figety or restless that you have been moving around a lot more than usual': "PHQ9_8",
    'Thoughts that you would be better off dead, or of hurting yourself': "PHQ9_9",
}
PHQ9_ANSWER_MAP = {
    "Not at all": 0,
    "Several days": 1,
    "More than half the days": 2,
    "Nearly every day": 3
}
selected_cols = [col for col in mapped.columns if col in PHQ9_MAP.keys()]
transformed_df = pd.melt(survey_data, id_vars=['user'], value_vars=selected_cols, var_name='question', value_name='raw_answer')
transformed_df['id'] = transformed_df['question'].replace(PHQ9_MAP.keys())

,user,question,raw_answer,id
0,u00,Little interest or pleasure in doing things,Not at all,Little interest or pleasure in doing things
1,u01,Little interest or pleasure in doing things,Several days,Little interest or pleasure in doing things
2,u02,Little interest or pleasure in doing things,More than half the days,Little interest or pleasure in doing things
3,u03,Little interest or pleasure in doing things,Not at all,Little interest or pleasure in doing things
4,u04,Little interest or pleasure in doing things,Several days,Little interest or pleasure in doing things
...,...,...,...,...
409,u53,"Thoughts that you would be better off dead, or...",Not at all,"Thoughts that you would be better off dead, or..."
410,u56,"Thoughts that you would be better off dead, or...",Not at all,"Thoughts that you would be better off dead, or..."
411,u57,"Thoughts that you would be better off dead, or...",Not at all,"Thoughts that you would be better off dead, or..."
412,u58,"Thoughts that you would be better off dead, or...",Not at all,"Thoughts that you would be better off dead, or..."


In [10]:
numerical_map = {
    "Not at all": 0,
    "Several days": 1,
    "More than half the days": 2,
    "Nearly every day": 3
}
def PHQ9_sum_to_group(sum):
    if sum < 5:
        return "minimal"
    elif sum < 10:
        return "mild"
    elif sum < 15:
        return "moderate"
    elif sum < 20:
        return "moderately severe"
    else:
        return "severe"
mapped = survey_data.copy()
cols = [col for col in mapped.columns if col not in ["user", "type", "Response"]]
for col in cols:
        mapped[col] = mapped[col].apply(lambda x: numerical_map[x])
mapped["sum"] = mapped[cols].sum(axis=1)
mapped["group"] = mapped["sum"].apply(PHQ9_sum_to_group)
mapped[["user", "group"]]

,user,group
0,u00,minimal
1,u01,mild
2,u02,moderate
3,u03,minimal
4,u04,mild
5,u05,minimal
6,u07,mild
7,u08,mild
8,u09,minimal
9,u10,minimal


,user,type,group
0,u00,pre,minimal
1,u01,pre,mild
2,u02,pre,moderate
3,u03,pre,minimal
4,u04,pre,mild
5,u05,pre,minimal
6,u07,pre,mild
7,u08,pre,mild
8,u09,pre,minimal
9,u10,pre,minimal


In [18]:
fig = countplot.countplot(data,
                          fig_title='Group level activity score distributions', 
                          plot_type='count', 
                          points='outliers',
                          aggregation='user', 
                          user=None,
                          column=' activity inference',
                          binning=False)

fig.show()

In [20]:
resampled = niimpy.util.aggregate(data, "1H")
resampled = resampled.reset_index("user")
fig = countplot.countplot(resampled, 
                          fig_title='Activity event counts by user', 
                          plot_type='count', 
                          points='all',
                          aggregation='user', 
                          user=None, 
                          column=None,
                          binning=False)
fig.show()

                           activity inference
user timestamp                               
u00  2013-03-27 04:00:00             0.000000
     2013-03-27 05:00:00             0.000000
     2013-03-27 06:00:00             0.060519
     2013-03-27 07:00:00             0.000000
     2013-03-27 08:00:00             0.000000
...                                       ...
u59  2013-05-31 23:00:00             2.330612
     2013-06-01 00:00:00             1.059190
     2013-06-01 01:00:00             0.835333
     2013-06-01 02:00:00             0.354628
     2013-06-01 03:00:00             0.004182

[72975 rows x 1 columns]
                         user  activity
user timestamp                         
u00  2013-03-27 04:00:00  u00       0.0
     2013-03-27 05:00:00  u00       0.0
     2013-03-27 06:00:00  u00       0.0
     2013-03-27 07:00:00  u00       0.0
     2013-03-27 08:00:00  u00       0.0
...                       ...       ...
u59  2013-05-31 23:00:00  u59       3.0
     2013-06-01 

In [1]:
import niimpy
niimpy.reading.read.read_mhealth_sleep_duration("mhealth_test.json")


{'sleep_duration': {'value': 7.5, 'unit': 'h'}, 'effective_time_frame': {'time_interval': {'start_date_time': '2016-02-05T20:35:00Z', 'end_date_time': '2016-02-06T06:35:00Z'}}}
